In [ ]:
import torch
torch.cuda.is_available()

### 개념 참고 링크
  
선형 레이어와 비선형 레이어  
https://go-hard.tistory.com/14  
배치사이즈(batch size),에포크(epoch),반복(iteration) 차이  
https://losskatsu.github.io/machine-learning/epoch-batch/# 

기본 연산 복습  
https://tutorials.pytorch.kr/beginner/blitz/tensor_tutorial.html   
학습률(Learning rate)  
https://bioinformaticsandme.tistory.com/130  




### Pytorch docs  
 

torch.nn.Sequential(*args)  
https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html  
torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)  
https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html?highlight=nn%20leakyrelu#torch.nn.LeakyReLU  
torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)  
https://pytorch.org/docs/stable/optim.html?highlight=optim%20adam#torch.optim.Adam  
torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')  
https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html  
 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms


from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard





# Descriminator 가 이미지가 진짜인지 가짜인지 판별 
# nn.Module 상속 
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            # 0.1 : 하이퍼 파라미터의 일종 
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
              # linear layer 256 to image dimension
            nn.Linear(256, img_dim), # mnist 이미지가 28x28x1사이즈 -> 786 
            # 픽셀 출력값들이 -1 에서 1 사이의 값임을 보장하기 위해 
            # mnist 데이터셋으로부터 입력을 정규화 
            # 입력값을 [-1,1] 로 하면 출력값도 [-1,1] 
            # 탄젠트 함수임
            nn.Tanh(),  
        )

    def forward(self, x):
        return self.gen(x)


# Hyperparameters etc.
device = "cuda" if torch.cuda.is_available() else "cpu"
print('gpu 사용여부:',device)

# learning rate 설정
# 
lr = 3e-4

# dimention : 64 로 설정  
# 입력 파라미터임 128,256 으로도 설정 가능 
z_dim = 64

# Gan 은 Hyperparameter 에 민감함 
# replication the original gan paper in a way 
image_dim = 28 * 28 * 1  # 784

# 배치사이즈를 32 로 설정 
batch_size = 32

# 에포크 50 번 돌림
num_epochs = 50

# discriminator 초기화 (init) 
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)


# noise 설정 
# 설정하는 이유 : 에포크마다 어떻게 변하는지 보기 위해 
fixed_noise = torch.randn((batch_size, z_dim)).to(device)

# the actual mean standard deviation for the mnist data set 
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,)),]
)


# MNIST 실습을 위해 파이토치로 데이터셋을 다운로드  
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)

# 데이터 로더 생성 
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 옵티마이저 정의 
opt_disc = optim.Adam(disc.parameters(), lr=lr)

# 제너레
opt_gen = optim.Adam(gen.parameters(), lr=lr)

# 
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f"logs/GAN_MNIST_fake")
writer_real = SummaryWriter(f"logs/GAN_MNIST_real")
step = 0

for epoch in range(num_epochs):
    # loader : 데이터 로더 
    # batch_idx ,real 
    print(loader) 
    for batch_idx, (real, _) in enumerate(loader):
        # real 
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        # criterion : loss func 
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        # where the second option of maximizing doesn't suffer from
        # saturating gradients
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1
                
%tensorboard --logdir logs/fit 

gpu 사용여부: cuda


In [1]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.
